In [13]:
import numpy as np
import pdb

In [4]:
with open('../node/finalizedPrices.txt','r') as fp:
    finalizedMarkets = eval(fp.read())

In [5]:
categoricalMarkets = [i for i in finalizedMarkets if i[0]['numOutcomes'] > 2]
binaryMarkets = [i for i in finalizedMarkets if i[0]['marketType'] == 'yesNo']

In [6]:
finalizedMarkets[0]

[{'id': '0x79ee0681a6090ac5169aaf04422daa8c5cccacb3',
  'universe': '0xe991247b78f937d7b69cfc00f1a487a293557677',
  'marketType': 'yesNo',
  'numOutcomes': 2,
  'minPrice': '0',
  'maxPrice': '1',
  'cumulativeScale': '1',
  'author': '0x9162a85e01a1863c9fa3549c5d83af1f587f1ac7',
  'consensus': {'isInvalid': 'false', 'payout': ['0', '10000']},
  'creationTime': 1532922190,
  'creationBlock': 6055019,
  'creationFee': '0.01',
  'settlementFee': '0.0101',
  'reportingFeeRate': '0.0001',
  'marketCreatorFeeRate': '0.01',
  'marketCreatorFeesBalance': '0',
  'marketCreatorMailbox': '0x9f5bd91bbe94a9e00f5e7f6b0cf89f4a98dd75e1',
  'marketCreatorMailboxOwner': '0x9162a85e01a1863c9fa3549c5d83af1f587f1ac7',
  'initialReportSize': '404871326465451718',
  'category': 'SPORTS',
  'tags': ['SOCCER', 'BARCELONA'],
  'volume': '98.47600655',
  'openInterest': '17.789',
  'outstandingShares': '17.789',
  'feeWindow': '0xdf4b6800f9e462edf43d7efb16b27334506afb4d',
  'endTime': 1540796400,
  'finalizatio

In [14]:
finalizedMarkets[0][1]

{'1': [{'price': '0.5', 'timestamp': 1533020047, 'amount': '1.5'},
  {'price': '0.4', 'timestamp': 1533021476, 'amount': '5'},
  {'price': '0.4851', 'timestamp': 1533058113, 'amount': '0.3'},
  {'price': '0.485', 'timestamp': 1533096308, 'amount': '4'},
  {'price': '0.4872', 'timestamp': 1533100267, 'amount': '4.8369'},
  {'price': '0.492', 'timestamp': 1533101807, 'amount': '4.8213'},
  {'price': '0.4722', 'timestamp': 1533103415, 'amount': '4.85'},
  {'price': '0.4692', 'timestamp': 1533108410, 'amount': '9.444'},
  {'price': '0.4781', 'timestamp': 1533109173, 'amount': '9.999'},
  {'price': '0.4691', 'timestamp': 1533110800, 'amount': '10'},
  {'price': '0.4691', 'timestamp': 1533111348, 'amount': '10'},
  {'price': '0.4726', 'timestamp': 1533111834, 'amount': '10'},
  {'price': '0.4726', 'timestamp': 1533138566, 'amount': '10'},
  {'price': '0.4683', 'timestamp': 1533139724, 'amount': '10'},
  {'price': '0.4649', 'timestamp': 1533140228, 'amount': '9'},
  {'price': '0.5554', 'times

In [8]:
admittedMarkets = finalizedMarkets
secondsAhead = 60 * 60 * 24 * 7 # 7 days

In [21]:
def gaussianKernel(x, y, sigma):
    return np.exp(-(x-y)**2/(2 * (sigma**2)))

def uniformKernel(x, y, halfWidth):
    if abs(x - y) <= halfWidth:
        return 1.
    else:
        return 0.

def logScore(probVector, outcomeIndex):
    return np.log(probVector[outcomeIndex])

def brierScore(probVector, outcomeIndex):
    if outcomeIndex == 0:
        return (1 - probVector[0]**2) + (0 - probVector[1]**2)
    else:
        return (0 - probVector[0]**2) + (1 - probVector[1]**2)
    
def sphericalScore(probVector, outcomeIndex):
    return probVector[outcomeIndex]/np.norm(probVector)

In [22]:
def scoreMarketsBinary(admittedMarkets, 
                       secondsAhead, 
                       kernelFunction, 
                       kernelWidth, 
                       scoreFunction, 
                       minTrades = 1, 
                       minWeight = 0.):
    
    weightVector = np.array([0])
    scoreVector = np.array([0])
    kernelWeightedVolume = 0
    allTrades = []
    for thisMarket in admittedMarkets:
        marketData = thisMarket[0]
        if len(list(thisMarket[1].keys())) < minTrades:
            continue
        else:
            pricedOutcome = list(thisMarket[1].keys())[0]
        priceData = thisMarket[1][pricedOutcome]
        marketId = marketData['id']
        endTime = marketData['endTime']
        consensusIndex = np.argmax(marketData['consensus']['payout'])
        centerTime = endTime - secondsAhead
        flipFlag = (int(pricedOutcome) != consensusIndex)
        for i,thisTrade in enumerate(priceData):
            
            # if the price of the designated outcome doesn't occur, flip the probability
            if not flipFlag:
                price = float(thisTrade['price'])
            else:
                price = 1. - float(thisTrade['price'])
                
            amount = float(thisTrade['amount'])
            tradeTime = thisTrade['timestamp']
            
            timeWeight = kernelFunction(centerTime, tradeTime, kernelWidth)
            weight = timeWeight * amount
            
            kernelWeightedVolume += timeWeight * amount
            allTrades.append((price, timeWeight * amount, flipFlag))
            if weight >= minWeight:
                weightVector = np.append(weightVector, weight)
                if consensusIndex == 0:
                    scoreVector = np.append(scoreVector,[scoreFunction([price, 1-price], 0)],0)
                else:
                    scoreVector = np.append(scoreVector,[scoreFunction([1-price, price],1)],0)
            
    return (np.average(scoreVector[1:],axis = 0,weights=weightVector[1:]), kernelWeightedVolume, allTrades)

In [24]:
_,_,allTrades = scoreMarketsBinary(binaryMarkets, 7 * 24 * 60 * 60, gaussianKernel, 24 * 60 * 60, logScore)

In [28]:
sorted(allTrades, key = lambda x: x[1], reverse=True)

[(0.75, 10.392218901517085, True),
 (0.73, 10.305028165668139, True),
 (0.6699999999999999, 5.4696325602836176, True),
 (0.75, 5.213309502626407, False),
 (0.65, 4.806254416598481, False),
 (0.46, 4.701719904130478, False),
 (0.6973, 4.554627756322638, False),
 (0.9, 3.36666748911084, False),
 (0.28500000000000003, 2.99988424521647, True),
 (0.28500000000000003, 2.6337968896237127, True),
 (0.6969, 2.120989479098965, False),
 (0.5, 2.066345578950235, False),
 (0.5, 2.027036449347451, False),
 (0.5, 1.9798465011832898, False),
 (0.85, 1.971396943362771, True),
 (0.5, 1.945232993666932, False),
 (0.55, 1.9249141366182585, False),
 (0.5, 1.9185084187221433, False),
 (0.5, 1.8654839715591311, False),
 (0.47, 1.7889902130699182, True),
 (0.845, 1.706368089323619, False),
 (0.7, 1.465428658581514, True),
 (0.75, 1.3198071836050216, True),
 (0.28500000000000003, 1.0491411136375062, True),
 (0.43, 0.9989607759215914, False),
 (0.98, 0.9914161748081365, True),
 (0.46, 0.9403439808260956, False)

In [ ]:
def confusionMatrixMake(allTrades):
    return 0